# Glucose Heatmap

This notebook generates a heatmap showing glucose levels by time of day from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Window size for smoothing the data
window = 5

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# Rename the existing date column, that contains a date and time and round the timestamp to the
# nearest 5 minutes to align the data (failing to do this means the pivot won"t work well when
# plotted)
df.rename(columns={ "date": "timestamp" }, inplace=True)
df["timestamp"] = df["timestamp"].dt.round("5min")

# Extract time of day and date
df["date"] = df["timestamp"].dt.date
df["time_str"] = df["timestamp"].dt.strftime("%H:%M")

# Use a rolling window to smooth the data
df["level_smooth"] = df["level"].rolling(window).mean()

# Pivot the data and sort into proper time order
heatmap_data = df.pivot(index="date", columns="time_str", values="level_smooth").sort_index()

# Preview the data
heatmap_data.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the heatmap
plt.figure(figsize=(12, 10))
ax = sns.heatmap(heatmap_data, cmap="coolwarm", center=6, cbar_kws={"label": "Glucose (mmol/L)"})

# Get current tick labels
xticks = ax.get_xticks()
xticklabels = ax.get_xticklabels()

# Show every other one to avoid clutter
ax.set_xticks(xticks[::2])
ax.set_xticklabels([label.get_text() for label in xticklabels[::2]], rotation=90)

# Set chart titles
plt.title("Glucose Heatmap by Time of Day")
plt.xlabel("Time of Day")
plt.ylabel("Date")
plt.xticks(rotation=90)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_heatmap", None, chart_export_format)

# Show the chart
plt.show()
